Імпорт необхідних бібліотек

hashlib - використовується для роботи з функцією хешування алгоритмом sha256

re - використовується для пошуку кількості нулів у префіксі хешу

time - використовується для визначення значення параметру timestamp у блоці

json - використовується для створення більш читабельного формату виведення інформації про блок на екран

In [5]:
import hashlib
import re
import time
import json

Допоміжна функція для пошуку кількості нулів у префіксі хешу

In [6]:
def getZeros(string):
    result = re.search(r"^0+", string)
    if result:
        return result.group(0)
    else:
        return ""

Клас BlockHeader, має функцію майнінгу та створення власного хешу. Функція майнінгу має задану умовою складність 4, працює з власним хешом блоку, що був створений на етапі ініціалізації інстансу класу і на його основі створює новий хеш блоку шляхом перебору nonce

In [7]:
class BlockHeader:
    version = 1
    merkleRoot = ""
    timestamp = 0
    bits = "ffff001f"
    nonce = 0
    blockHash = ""
    prevBlockHash = ""
    
    def __init__(self, prevBlockHash):
        self.prevBlockHash = prevBlockHash
        self.timestamp = int(time.time())
        self.blockHash = self.makeHash()

    def mine(self):
        difficultyTarget = 4
        while len(getZeros(self.blockHash)) != difficultyTarget:
            self.nonce += 1
            self.blockHash = self.makeHash()
        

    def makeHash(self):
        preEncodeString = f"{self.version}{self.merkleRoot}{self.timestamp}{self.nonce}{self.prevBlockHash}"
        hashRoundOne = hashlib.sha256(preEncodeString.encode('utf-8')).hexdigest()
        hashRoundTwo = hashlib.sha256(hashRoundOne.encode('utf-8')).hexdigest()
        return str(hashRoundTwo)


Основний клас Block, містить функції майнінгу, функцію повернення інформації про блок та допоміжну функцію для створення більш читабельного формату виводу інформації на екран

In [8]:
class Block:
    blockHeader = ""
    blockSize = 1
    txCount = 0
    txs = ""
    height = 0
    
    def __init__(self, height, prevBlockHash):
        self.blockHeader = BlockHeader(prevBlockHash)
        self.height = height

    def mine(self):
        self.blockHeader.mine();
        self.txs = f"Andrii sent {self.height} coins to Alice"
        self.txCount += 1
        self.blockHeader.merkleRoot = str(hashlib.sha256(self.txs.encode('utf-8')).hexdigest())
        return self
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    def getBlockData(self):
        print(self.toJSON())
        return self

Клас Blockchain, містить масив блоків chain, функції створення стартового блоку, для якого хеш попереднього блоку є нульовою строкою, та функцію створення нових блоків з їх додаванням до масиву. Нові блоки створюються з посиланням на попередні: для цього використовується висота попереднього блоку та його хеш. Нові блоки майняться та зберігаються до масиву

In [9]:
class Blockchain:
    chain = []

    def genesisBlock(self):
        zeroHash = "".rjust(64, "0")
        initialBlock = Block(0, zeroHash).mine().getBlockData()
        self.chain.append(initialBlock)
        return self

    def addBlock(self):
        previousBlock = self.chain[len(self.chain) - 1];
        newBlock = Block(previousBlock.height + 1, previousBlock.blockHeader.blockHash).mine().getBlockData();

        self.chain.append(newBlock);
        return self

Створення інстансу класу Blockchain, за його допомогою створення стартового блоку та додавання двох нових блоків. У процесі на екран виводиться інформація про кожний блок

In [10]:
blockchainInstance = Blockchain()
genesisBlock = blockchainInstance.genesisBlock()
genesisBlock.addBlock()
genesisBlock.addBlock()

{
    "blockHeader": {
        "blockHash": "0000697c46c8724b5db31285e086cfad85e301d25ad4fdaa1bbc8713161ff4c9",
        "merkleRoot": "440c15781fe1fd99a6b57d72d34ffb21d10a629e5e9dd69d1fe34bcd9f21affd",
        "nonce": 85041,
        "prevBlockHash": "0000000000000000000000000000000000000000000000000000000000000000",
        "timestamp": 1670186886
    },
    "height": 0,
    "txCount": 1,
    "txs": "Andrii sent 0 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "0000db673516e4dfa405b5a300629a18b93d8046a90d0163599d10852db5276d",
        "merkleRoot": "550394aef6271f3cfb36e5b25f0bbff2fd2f8369522b9853518573c9a910e189",
        "nonce": 78588,
        "prevBlockHash": "0000697c46c8724b5db31285e086cfad85e301d25ad4fdaa1bbc8713161ff4c9",
        "timestamp": 1670186886
    },
    "height": 1,
    "txCount": 1,
    "txs": "Andrii sent 1 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "0000bf6f0978de74736c665a2c220baed4784620db0f4b5a22eb3644f319165d",
        "merkl